# Exploring dimensionless numbers and Neural Network Regression
Author: Payam Mousavi  
Last updated: March 6, 2024 

Given the analytical equation (fitted empirically) for the Drag Coefficient $C_D$ provided in reference: https://pages.mtu.edu/~fmorriso/DataCorrelationForSphereDrag2016.pdf, we see that it's only a function of the Reynolds number, $Re = \rho.U.D/\mu$. Let's say that we did not know this ahead of time and wanted to perform experiments for a range of input parameters, namely, $\rho$, $U$, $D$, and $\mu$. Clearly, many combinations will be redundant since they correpond to the same $Re$. Consider the following models:  

Given $\rho$, $U$, $D$, and $\mu$, consider the following three models:  


<img src="./Figures/Model_0.png" width="300" height="200" /> . 
<img src="./Figures/Model_I.png" width="300" height="200" /> . 
<img src="./Figures/Model_II.png" width="300" height="200" /> . 

To start, we are interested in the following questions:  
1. Model_0, we fit just to verify that it's an easy regression problem. Note that, given the very large range of inputs spanning multiple orders of magnitute, this is not a trivial problem. So we might choose to limit the range. For example, we expect a linear function for low Re (i.e., Stokes flow), so we could ignore that range. 
2. Assuming we keep the number of trainable parameters approximately the same, is Model_I more data efficient than Model_II, if we randomly sample the 4 parameters?  Intuitively, we expect this to be true since we are biasing the structure of the neural network to discover the existence of an effective compression of the inputs (by $Re$).  
3. After training Model_I, does the single node actually correspond to $Re$? Can the model 'discover' the non-dimensional parameters?  If not, can we somehow help the model to discover this?  
4. (Optional) can we use the knowledge of the $Re$ to perform data augmentation thereby increasing the efficiency of the training? For example, given a combination of inputs, we could generate $n$ additional training samples corresponding to the same $Re$. This 'synthetic data' is expected to improve the efficiency. It is not clear whether this is useful in practice. Still interesting to explore.  


What's on the docket:  
* Verify that MSELoss from PyTorch is consistent with my manual calculation. **DONE**  
* Implement Model_0 to take Re as input and output $C_D$.  
* Implement Model_I to take the 4 parameters, bottleneck to Re, then output $C_D$. Is the Re node equal to the value of Re?  
* Quantify data requirements for each model to get similar accuracy? Do we need a hyperparameter optimization for this?  
* Move some sections out of the notebook for more flexibility:
    * Move dataset creation and loading 
    * Move helper functions to utils 
    * Move models 
* Tricks to improve fitting: scale the 4 inputs so that they are roughly in the same range.  
* Limit the range of interest to only include regions with interesting features.  
    






In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2


from utils import *
from models import Model_II, Model_0
from datasets import *
from train import train_model
from predict import predict_model
from eval import eval_model

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
from tqdm import tqdm
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam
import torch.nn.init as init
from torch.utils.tensorboard import SummaryWriter

matplotlib.rcParams.update({'font.size': 18})
eps = 1e-12

In [ ]:
# Re_vec = np.logspace(2, 7, 1000)
# rho, mu, D, U = generate_inputs_from_Re(Re_vec=Re_vec, u_range=[0.1, 1e3], rho_range=[1, 1000], mu_range=[1e-3, 1e3], D_range=[1e-3, 1e3])
# # rho# 

In [ ]:
def plot_CD(Re, CD, marker='o'):
    sns.set(style="whitegrid")
    plt.figure(figsize=(6, 6))
    sns.scatterplot(x=Re, y=CD, color='blue', marker=marker, alpha=0.5)
    # plt.xscale('log')
    # plt.yscale('log')
    plt.xlabel('$Re$')
    plt.ylabel('$C_D$')

In [ ]:
Re_vec = np.logspace(2, 7, 200)
# Re_vec = np.linspace(1e1, 1e7, 1000)
CDs, _ = run_experiments(Re_vec=Re_vec)

Re_vec_rescaled = Re_vec/1e6
plot_CD(Re_vec_rescaled, CDs, marker='o')

## Dataset Creation 
NOTE: Only run the dataset for the model of interest. Don't run both.

### 4-input dataset:

In [ ]:
# Create the dataloader/dataset:
# dataset = RandomDataset(num_samples=50000,
#                         Re_range=[1e4, 1e8],
#                         rho_range=[100, 3000],
#                         mu_range=[0.3e-3, 0.1],
#                         D_range=[0.05, 1],
#                         U_range=[1, 10],
#                         seed=1234)

dataset = RandomDataset_scaled(num_samples=50000,
                               parms_scaling=[1e3, 1e-2, 1, 1],
                               Re_range=[1e4, 1e8],
                               rho_range=[100, 3000],
                               mu_range=[0.3e-3, 0.1],
                               D_range=[0.05, 1],
                               U_range=[1, 10],
                               seed=1234)


# Split the dataset into train, validation, and test sets:
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True, num_workers=3)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False, num_workers=3)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False, num_workers=3)

# Extract Re values for test set:
train_Re = extract_Re_values(dataset, train_dataset)
val_Re = extract_Re_values(dataset, val_dataset)
test_Re = extract_Re_values(dataset, test_dataset)

In [ ]:
max(train_Re)

### 1-input dataset:

In [ ]:
# Create the dataloader/dataset:
dataset = RandomDataset_Re(num_samples=50000,
                           Re_range=[1e4, 1e7],
                           re_scaling=1e6,
                           seed=123)


# Split the dataset into train, validation, and test sets:
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False, num_workers=0)

# Extract Re values for test set (used for visualizing histograms):
test_Re = []
for re, _ in test_dataset:
    test_Re.append(re.item())

test_Re = np.array(test_Re)

In [ ]:
# rho, mu, D, U, Re_vec, CD = sample_parameters(num_samples=1000, 
#                                               Re_range=[1e0, 3e7], 
#                                               rho_range=[50, 3000], 
#                                               mu_range=[0.3e-3, 5], 
#                                               D_range=[0.05, 10], 
#                                               U_range=[.001, 10])

# plt.loglog(Re_vec, CD, 'o', alpha=0.5)

In [ ]:
plt.hist(test_Re, bins=10, edgecolor='black')
plt.xlabel('Reynolds Number (Re)')
plt.ylabel('Frequency')
plt.title('Histogram of Reynolds Numbers in Test Dataset')
plt.show()

min(test_Re), max(test_Re)

# Training

### Model_II:

In [ ]:
# Setting up the model:
HIDDEN_DIMS = [32, 32, 16, 8, 4]
MODEL_NAME = 'Model_II'
MODEL = Model_II(input_dim=4, output_dim=1, hidden_dims=HIDDEN_DIMS).float()
CRITERION = nn.MSELoss() # criterion = nn.L1Loss(), nn.SmoothL1Loss(), ...
OPTIMIZER = Adam(MODEL.parameters(), lr=1e-3)
WRITER = SummaryWriter('runs/' + MODEL_NAME)
MODEL_SAVE_PATH = 'models/' + MODEL_NAME + '.pt'
WRITER_PATH = 'runs/' + MODEL_NAME

NUM_EPOCHS = 500

num_trainable_params = sum(p.numel() for p in MODEL.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {num_trainable_params}")

### Model_0:

In [ ]:
# Setting up the model:
HIDDEN_DIMS = [64, 32, 16, 8, 4]
MODEL_NAME = 'Model_0'
MODEL = Model_0(input_dim=1, output_dim=1, hidden_dims=HIDDEN_DIMS).float()
CRITERION = nn.MSELoss() # criterion = nn.L1Loss(), nn.SmoothL1Loss(), ...
OPTIMIZER = Adam(MODEL.parameters(), lr=1e-3)
WRITER = SummaryWriter('runs/' + MODEL_NAME)
MODEL_SAVE_PATH = 'models/' + MODEL_NAME + '.pt'
WRITER_PATH = 'runs/' + MODEL_NAME

NUM_EPOCHS = 500

num_trainable_params = sum(p.numel() for p in MODEL.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {num_trainable_params}")

In [ ]:
best_model_state, history = train_model(model=MODEL, 
                                        train_loader=train_loader, 
                                        val_loader=val_loader, 
                                        test_loader=test_loader, 
                                        optimizer=OPTIMIZER, 
                                        criterion=CRITERION, 
                                        num_epochs=NUM_EPOCHS, 
                                        model_save_path=MODEL_SAVE_PATH,
                                        writer_path=WRITER_PATH)

In [ ]:
# Prediction:
DATASET_TEST = test_dataset
MODEL_SAVE_PATH = 'models/' + MODEL_NAME + '.pt'
predictions = predict_model(model_path=MODEL_SAVE_PATH, 
                            dataset=DATASET_TEST)

# Evaluation:
mse, mae = eval_model(DATASET_TEST, MODEL_SAVE_PATH, visualize=True)
print(f"MSE: {mse:.5f}, MAE: {mae:.5f}")


## Inference & Testing:  
Visualize the model fit by looking at the curves for a regularly-spaced Re vector covering the entire range.

In [ ]:
def infer_cd_from_re(re_vector, model_path, hidden_dims = HIDDEN_DIMS, inputs=None):
    # HIDDEN_DIMS = [256, 256, 64, 32] #TODO: add this as a parameter to this function
    model = Model_II(input_dim=4, output_dim=1, hidden_dims=[256, 128, 64, 32, 32, 32, 16, 8, 4])
    model_checkpoint = torch.load(model_path)
    model.load_state_dict(model_checkpoint)
    model.eval()

    # Check if inputs are provided or need to be generated
    if inputs is None:
        # Generate inputs from Re vector
        rho, mu, D, U = generate_inputs_from_Re(Re_vec=re_vector,  # Corrected variable name
                                                u_range=[0.01, 20],
                                                rho_range=[10, 2000],
                                                mu_range=[1e-3, 0.1],
                                                D_range=[0.05, 0.5])
        input_tensor = torch.tensor(np.column_stack((rho, mu, D, U)), dtype=torch.float32)
    else:
        test_inputs = []
        for idx in range(len(inputs)):
            input_sample, _ = inputs[idx]  # Get the input sample and ignore the target
            test_inputs.append(input_sample)  # Convert to numpy if it's a tensor

        # Convert list to numpy array and then to tensor
        input_tensor = torch.tensor(np.stack(test_inputs), dtype=torch.float32)

    # Perform inference
    with torch.no_grad():
        cd_predicted = model(input_tensor).squeeze().numpy()

    return cd_predicted



### Method #1:

In [ ]:
rho, mu, D, U, Re_vec, CD = sample_parameters_naive(num_samples=5000,
                                              Re_range=None,
                                              rho_range=[10, 2000],
                                              mu_range=[0.001, 0.1],
                                              D_range=[0.05, 0.5],
                                              U_range=[0.01, 20])

cd_predicted = infer_cd_from_re(re_vector=Re_vec, model_path='models/Model-II-checkpoint.pth')
cd_true = CD

plt.figure(figsize=(10, 6))

plt.scatter(Re_vec, cd_predicted, color='red',
            label='CD Predicted', marker='x')
plt.scatter(Re_vec, cd_true, color='blue', label='CD True', marker='o')
plt.xscale('log')
plt.yscale('log')


plt.xlabel('Reynolds Number')
plt.ylabel('Drag Coefficient (CD)')
plt.title('Comparison of True and Predicted CD values')
plt.legend()
plt.show()

mse_error = np.mean((cd_true - cd_predicted)**2)
print("MSE:", mse_error)

In [ ]:
Re_vec[0:10]

### Method #2:

In [ ]:
# Testing:
# Generate a random Re vector:
Re_vec = np.array(train_Re) #np.logspace(2, 7, 200)
# Re_vec = np.logspace(-1, 11, 5000)
# Infer the CD from the Re vector:
cd_predicted = infer_cd_from_re(Re_vec, model_path='models/Model-II-checkpoint.pth', inputs=train_dataset)
cd_true, _ = run_experiments(Re_vec=Re_vec)

plt.figure(figsize=(10, 6))

# Plotting predicted CD values
plt.scatter(Re_vec, cd_predicted, color='red', label='CD Predicted', marker='x', alpha=0.5)
plt.scatter(Re_vec, cd_true, color='blue', label='CD True', marker='o', alpha=0.5)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Reynolds Number')
plt.ylabel('Drag Coefficient (CD)')
plt.title('Comparison of True and Predicted CD values')
plt.legend()
plt.show()


mse_error = np.mean((cd_true - cd_predicted)**2)
print("MSE:", mse_error)
r2 = 1 - np.sum((cd_true - cd_predicted)**2) / np.sum((cd_true - np.mean(cd_true))**2)
print("R2:", r2)


plt.hist(Re_vec, bins=10, edgecolor='black')
plt.xlabel('Reynolds Number (Re)')
plt.ylabel('Frequency')
plt.title('Histogram of Reynolds Numbers in Test Dataset')
plt.show()


In [ ]:
min(Re_vec)